# Ambulatory Glucose Profile (AGP)

This notebook generates an AGP chart from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import datetime

# Reporting date range
start = datetime(2025, 1, 1, 0, 0, 0)
end = datetime(2025, 1, 1, 23, 59, 59)

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Rolling window size for smoothing the AGP chart using the rolling median method
window = 3

# Fraction of the total number of points used to fit each local regression for the LOWESS chart.
# Larger values give a smoother curve
fraction = 0.05
iterations = 3

# Set the interquartile ranges
lower_percentile = 0
upper_percentile = 100

# Set the Y-axis limits
y_min = 3
y_max = 15

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

# Create a "seconds since midnight" column to enable plotting
df["time_minutes"] = df["date"].dt.hour * 60 + df["date"].dt.minute

# Group by time since midnight
grouped_df = df.groupby("time_minutes")["level"]

# Calculate percentiles
agp = grouped_df.agg([
    ("median", "median"),
    (r"lower_percentile", lambda x: np.percentile(x, lower_percentile)),
    (r"upper_percentile", lambda x: np.percentile(x, upper_percentile)),
])

# Smooth the data using a rolling median
agp = agp.rolling(window=window, center=True, min_periods=1).median()

# Smooth the data using LOWESS smoothing
agp["median_lowess"] = lowess(agp["median"], agp.index, frac=fraction, it=iterations, return_sorted=False)
agp["lower_percentile_lowess"] = lowess(agp["lower_percentile"], agp.index, frac=fraction, it=iterations, return_sorted=False)
agp["upper_percentile_lowess"] = lowess(agp["upper_percentile"], agp.index, frac=fraction, it=iterations, return_sorted=False)

# Then clip the median to the interquartile range
agp["median_smooth_clipped"] = np.clip(agp["median_lowess"], agp["lower_percentile_lowess"], agp["upper_percentile_lowess"])

# Preview the data
agp.head()

In [ ]:
# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet("glucose_agp", { "Trend": agp })

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

# Shaded area between 10th and 90th percentile
plt.fill_between(
    agp.index, 
    agp["lower_percentile"], 
    agp["upper_percentile"], 
    color="orange", 
    alpha=0.4, 
    label=f"Interquartile Range ({lower_percentile}%-{upper_percentile}%)"
)

# Median line
plt.plot(
    agp.index, 
    agp["median"], 
    color="red", 
    label="Median"
)

# Optional: target glucose range
plt.axhline(3.9, color="red", linestyle="--", label="Lower Target (3.9 mmol/L)")
plt.axhline(10.0, color="green", linestyle="--", label="Upper Target (10.0 mmol/L)")

plt.title("Ambulatory Glucose Profile (AGP)")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.grid(True)
plt.xlim(0, 1440)
plt.xticks(np.arange(0, 1441, 60), labels=[f"{h}:00" for h in range(25)])

# Move the legend below the plot
plt.legend(
    loc="upper center", 
    bbox_to_anchor=(0.5, -0.15), 
    ncol=2,   # number of columns
    frameon=False
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_agp", "Rolling-Median", chart_export_format)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

# Shaded area between 10th and 90th percentile
plt.fill_between(
    agp.index, 
    agp["lower_percentile_lowess"], 
    agp["upper_percentile_lowess"], 
    color="orange", 
    alpha=0.4, 
    label=f"Interquartile Range ({lower_percentile}%-{upper_percentile}%)"
)

# Median line
plt.plot(
    agp.index, 
    agp["median_lowess"], 
    color="red", 
    label="Median"
)

# Target glucose range
plt.axhline(3.9, color="red", linestyle="--", label="Lower Target (3.9 mmol/L)")
plt.axhline(10.0, color="green", linestyle="--", label="Upper Target (10.0 mmol/L)")

plt.title("Ambulatory Glucose Profile (AGP)")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.grid(True)
plt.xlim(0, 1440)
plt.xticks(np.arange(0, 1441, 60), labels=[f"{h}:00" for h in range(25)])

# Move the legend below the plot
plt.legend(
    loc="upper center", 
    bbox_to_anchor=(0.5, -0.15), 
    ncol=2,   # number of columns
    frameon=False
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_agp", "LOWESS", chart_export_format)

# Show the plot
plt.show()